# Semantic Search and Retrieval-Augmented Generation (RAG) 
1. Drop the dataset if it is already exists
2. Create the database
3. Use that database
4. Inside that database create table called 'video_game_wikipedia'
5. define the tables with attributes

In [10]:
%%sql
drop database if exists video_game_wikipedia;
create database video_game_wikipedia;
use video_game_wikipedia;

create table vecs(
    id bigint(20),
    url text default null,
    paragraph text default null,
    v vector(1536) not null,
    shard key(id),
    key(id) using hash,
    fulltext (paragraph)
);

1 rows affected.

1 rows affected.

++
||
++
++

## Function that produces Random floats of specified range

In [11]:
%%sql
create or replace function randbetween(a float, b float) returns float
as
begin
    return (rand()*(b-a) + a);
end;

1 rows affected.

++
||
++
++

## Using random floats, creating a function to generate a vector of specified length

In [12]:
%%sql
set sql_mode = pipes_as_concat;

create or replace function gen_vector(length int) returns text as
declare s text = "[";
begin 
    if length < 2 then
        raise user_exception("length too short: " || length);
    end if;

    for i in 1..length-1 loop
        s = s || randbetween(-1,1) || ",";
    end loop;
    s = s || randbetween(-1,1) || "]";
    return s;
end;

1 rows affected.

++
||
++
++

## Function to normalize vectors specifically with dimension (1536)

In [13]:
%%sql
create or replace function normalize(v blob) returns blob as
declare 
    squares blob = vector_mul(v,v);
    length float = sqrt(vector_elements_sum(squares));
begin
    return scalar_vector_mul(1/length, v);
end;

1 rows affected.

++
||
++
++

In [14]:
%%sql
create or replace function norm1536(v vector(1536)) returns vector(1536) as 
begin
    return normalize(v) :> vector(1536);
end;

1 rows affected.

++
||
++
++

In [19]:
%%sql
create or replace function nrandv1536() returns vector(1536) as 
begin
    return norm1536(gen_vector(1536));
end;

1 rows affected.

++
||
++
++

## Populate the vecs table with 10 Million mock vectors

In [15]:
%%sql
create or replace procedure insert_vectors(num_rows bigint) as 
declare c int;
begin
    select count(*) into c from vecs;
    loop
        insert into vecs (id, v)
        select id + (select max(id) from vecs), nrandv1536()
        from vecs
        where id <= 128 * 1024;
        select count(*) into c from vecs;
        if c >= num_rows then
            exit;
        end if;
    end loop;
end;

1 rows affected.

++
||
++
++

## Populate the Table

In [20]:
%%sql
insert into vecs (id, v) values (1,nrandv1536());
call insert_vectors(10000000);

1 rows affected.

++
||
++
++

Importing Real Wikipedia Video Game data into Vecs table using Single Store Pipeline (enrich our mock vectors with actual content) 

In SingleStore (and MySQL-compatible dialects), identifiers such as table names and pipeline names should use backticks (`), not single quotes.

1) Create a pipeline
2) import from S3 buckets to vecs table
3) Data is CSV in that URL
4) Hence the S3 bucket is public we left the aws_access_key_id & aws_secret_access_key empty
5) format of csv file with delimiter, line termination characters, ensuring the data is parsed correctly

In [43]:
%%sql
create or replace pipeline `wiki_pipeline` as 
load data S3 's3://wikipedia-video-game-data/video-game-embeddings(1).csv'
config '{"region":"us-west-1"}'
credentials '{"aws_access_key_id": "",
            "aws_secret_access_key": ""}'
skip duplicate key errors
into table `vecs`
format csv
fields terminated by ','
enclosed by '"'
lines terminated by '\r\n';


++
||
++
++

## Start the pipeline to import the data

In [44]:
%%sql
start pipeline `wiki_pipeline`;

++
||
++
++

## Monitoring the pipeline to ensure the data is loaded 

In [45]:
%%sql
select DATABASE_NAME, PIPELINE_NAME, BATCH_ID, BATCH_STATE, START_TIME, ROWS_STREAMED, ROWS_PER_SEC
from information_schema.PIPELINES_BATCHES_SUMMARY
order by BATCH_ID;

1 rows affected.

DATABASE_NAME,PIPELINE_NAME,BATCH_ID,BATCH_STATE,START_TIME,ROWS_STREAMED,ROWS_PER_SEC
video_game_wikipedia,wiki_pipeline,2,In Progress,2025-05-26 07:24:06.535523,466,177.65747562451747


## Building Vector Indices

1) Single Store Offers options to create vector indices
2) Each with various tunable parameters
3) IVF - Inverted File Indices

starting with automatic vector index, automatic index in V column, SingleStore chooses best indices based on the data  

In [6]:
%%sql

alter table vecs add vector index auto (v) INDEX_OPTIONS '{"index_type":"AUTO"}';

++
||
++
++

## Create Index using IVF flat method 

IVF Flat is commonly used index type that organizes vector into clusters allowing for efi=ficient searches by narrowing down the search space to specific clusters

In [26]:
%%sql

alter table vecs add vector index ivf_flat (v) INDEX_OPTIONS '{"index_type":"IVF_FLAT"}';

++
||
++
++

In [8]:
%%sql
alter table vecs add vector index ivf_pq (v) INDEX_OPTIONS '{"index_type":"IVF_PQ"}'; 

++
||
++
++

ivf-pq - inverted file with product qunatisation is advanced indexing method that further compresses the data, trading off some of the accuracy but we get much faster search time and reduced storage requirements

In [27]:
%%sql
show INDEXES from vecs;

6 rows affected.

Table,Non_unique,Key_name,Seq_in_index,Column_name,Collation,Cardinality,Sub_part,Packed,Null,Index_type,Comment,Index_comment,Index_options
vecs,1,__SHARDKEY,1,id,None,None,None,None,YES,METADATA_ONLY,,,
vecs,1,id,1,id,None,None,None,None,YES,COLUMNSTORE HASH,,,
vecs,1,paragraph,1,paragraph,utf8mb4_general_ci,None,None,None,YES,FULLTEXT,,,
vecs,1,auto,1,v,None,None,None,None,,VECTOR,,,"{""index_type"":""AUTO""}"
vecs,1,ivf_pq,1,v,None,None,None,None,,VECTOR,,,"{""index_type"":""IVF_PQ""}"
vecs,1,ivf_flat,1,v,None,None,None,None,,VECTOR,,,"{""index_type"":""IVF_FLAT""}"


## Testing Indices 

ANN - Approximate Nearest Neighbours
KNN - K Nearest Neighbours - Average of K Closest points

Testing a vector corresponding to the paragraph Nintendo's Rad Racer video game and testing how we are retrieving similiar paragraphs using different indexing methods   

In [19]:
%%sql
set @qv = (select v from vecs where id = 1125899906845489);

select paragraph, v <*> @qv as sim
from vecs
order by sim use index () desc
limit 5;

5 rows affected.

paragraph,sim
"Rad Racer was ranked number 57 on IGN's Top 100 Nintendo Entertainment System games and was called ""iconic"" and one of the NES's premier racing games. Maxim included the game amongst thirteen others in their greatest 8-bit video games of all time list.",1.0000001192092896
"Rad Racer was met with favorable reviews, enjoyed commercial success, and sold 1.96 million copies. It also ranked 8th on Nintendo Power's player's poll Top 30. Famitsu praised the sense of speed but felt the game was slightly monotonous. Japanese publication Family Computer Magazine applauded the variety of game landscapes found in different levels. British magazine Computer and Video Games called it an ""extremely playable racing game"" and said ""things get very fast and competitive as you get further into the game.""",0.9292747974395752
"The game sold 1.96 million copies and is considered one of the best racing games on the NES, but was criticized as being derivative of other racing games from the period. Reviewers widely compared the game to Out Run, though opined that Rad Racer was different in some ways, and they praised the sense of speed. The game appeared in the 1989 film The Wizard and was one of three games to feature a unique competition course in the 1990 Nintendo World Championship.",0.9212067723274231
"Rad Racer was released in Japan on August 7, 1987. It was later released in North America in October 1987 and in Europe on January 15, 1988. In August 1987, Nintendo released an arcade machine called Playchoice-10 that contained up to ten popular NES titles. Amongst the eligible NES games, Rad Racer was also available on the arcade machine.",0.8954247236251831
"Due to most of Rad Racer's sales being from the United States, Rad Racer II was developed and released only in North America for the NES and the arcade. The sequel featured eight new tracks and new music but similar gameplay. Hardcore Gamer 101 said that steering was looser than the first game and rival cars more aggressive, leading to a less enjoyable playing experience. Despite the efforts of Square to make unique games with 3D features such as Rad Racer and 3-D Worldrunner, and high sales, the company was in financial trouble. These events are what led to a final attempt at a breakout hit, Final Fantasy. Rad Racer appeared in a scene in the movie The Wizard. It was also one of three games, including Super Mario Bros. and Tetris, featured at the 1990 Nintendo World Championship with an exclusive racing level to complete as one of the rounds of competition. The limited-edition release of the game's cartridge used in the tournament is now the rarest and most valuable Nintendo games available.",0.8908036351203918


Searchs through all the vectors without any index (Brute Force Search)

### Testing Performance using Auto Index
Single Store will optimize the Search, gives significant Speed Up when compared to KNN Search

In [21]:
%%sql
set @qv = (select v from vecs where id = 1125899906845489);

select paragraph, v <*> @qv as sim
from vecs
order by sim use index (auto) desc
limit 5;

5 rows affected.

paragraph,sim
"The game sold 1.96 million copies and is considered one of the best racing games on the NES, but was criticized as being derivative of other racing games from the period. Reviewers widely compared the game to Out Run, though opined that Rad Racer was different in some ways, and they praised the sense of speed. The game appeared in the 1989 film The Wizard and was one of three games to feature a unique competition course in the 1990 Nintendo World Championship.",0.9212067723274231
"Moto Racer Advance was first displayed at the 2002 Electronic Entertainment Expo . IGN gave the game two awards for portable games after its E3 2002 coverage: ""Best Graphics"" and ""Best Racing Game"". IGN praised the early version of the game for its high draw distance and smooth frame rate. The game was built from the ground up to be a racing game for the Game Boy Advance and to take advantage of the hardware offered by the system. IGN previewed the game six months before it was made available for retail and called the graphics the game's highlight, while noting that tune-ups in the physics engine were needed before release. The game manipulated 2D sprites and backgrounds to give the impression of 3D to the player. By keeping the core graphics engine simple, the game was able to contain elongated draw distances and a smooth frame rate.",0.8628146648406982
"In 1997, Electronic Gaming Monthly ranked it the 56th best console video game of all time. They explained their decision to include it on the list instead of Punch-Out!!: ""The NES version is great, but the Super NES one is even better, with large, well-animated characters, great control and a near-perfect learning curve."" In 2018, Complex rated the game number 18 on their ""The Best Super Nintendo Games of All Time."" 1995, Total! placed the game 27th in their Top 100 SNES Games.",0.8584308624267578
"Overall assessments of the game were mostly positive. GamePro gave it a 4.5 out of 5 for sound and a perfect 5.0 in control, graphics, and fun factor, calling it ""a feverishly fun Nintendo 64 racer that combines elements of Mario Kart 64, Wave Race 64, and Pilotwings 64 into one spectacular game."" EGM named it ""Game of the Month"", with its four reviewers lauding the challenging gameplay and numerous objectives to tackle. Gerstmann instead counted the latter as the game's greatest weakness, arguing that having to repeatedly play through the same courses with slightly different objectives makes the game excessively repetitive. He concluded that the game is far better than Mario Kart 64, but the repetitiveness ""ultimately kills it."" Next Generation fell more in line with the majority, remarking that the combination of racing and adventure elements works well, and that ""Diddy Kong Racing shows Rare's pure craftmanship, displaying keen subtleties that eventually win players over.""",0.8548850417137146
"Similar compliments were made of the other versions. A reviewer from Consoles + praised the SNES version's simultaneous four-player function and the addition of game modes and tracks. Joypad 's reviewer praised the multiplayer, saying it is ""simply excellent"", and gave the vehicles' manoeuvrability a similar description. Power Play's reviewer praised the MS-DOS version's variety. Vince Broady of GameSpot praised the ""excellent"" gameplay, but complained that the CD soundtrack is repetitive, and that the graphics were not as good as competitors. Coming Soon Magazine's reviewer complimented its ""terrific"" gameplay and praised the track editor, saying it was ""a tremendous idea"" and that it increases playability. Steve Bauman of Computer Games Strategy Plus had mixed feelings: he believed the game is ""shallow as hell"", but also described it as ""surprisingly entertaining"". PC Zone's Charlie Brooker liked the graphics and sound, describing them as ""cute"" and ""neat"" respectively, and praised the game's ""timeless console-style action"". The game was named a PC Zone classic. 

### Testing Performance using IVF FLAT
IVF FLAT Index further enhances the performance by clustering the vectors and searching within the most relevant clusters offering faster results than the exact search.

In [29]:
%%sql
set @qv = (select v from vecs where id = 1125899906845489);

select paragraph, v <*> @qv as sim
from vecs
order by sim use index (ivf_flat) desc
limit 5;

5 rows affected.

paragraph,sim
"Rad Racer was ranked number 57 on IGN's Top 100 Nintendo Entertainment System games and was called ""iconic"" and one of the NES's premier racing games. Maxim included the game amongst thirteen others in their greatest 8-bit video games of all time list.",1.0000001192092896
"Rad Racer was met with favorable reviews, enjoyed commercial success, and sold 1.96 million copies. It also ranked 8th on Nintendo Power's player's poll Top 30. Famitsu praised the sense of speed but felt the game was slightly monotonous. Japanese publication Family Computer Magazine applauded the variety of game landscapes found in different levels. British magazine Computer and Video Games called it an ""extremely playable racing game"" and said ""things get very fast and competitive as you get further into the game.""",0.9292747974395752
"The game sold 1.96 million copies and is considered one of the best racing games on the NES, but was criticized as being derivative of other racing games from the period. Reviewers widely compared the game to Out Run, though opined that Rad Racer was different in some ways, and they praised the sense of speed. The game appeared in the 1989 film The Wizard and was one of three games to feature a unique competition course in the 1990 Nintendo World Championship.",0.9212067723274231
"Rad Racer was released in Japan on August 7, 1987. It was later released in North America in October 1987 and in Europe on January 15, 1988. In August 1987, Nintendo released an arcade machine called Playchoice-10 that contained up to ten popular NES titles. Amongst the eligible NES games, Rad Racer was also available on the arcade machine.",0.8954247236251831
"Due to most of Rad Racer's sales being from the United States, Rad Racer II was developed and released only in North America for the NES and the arcade. The sequel featured eight new tracks and new music but similar gameplay. Hardcore Gamer 101 said that steering was looser than the first game and rival cars more aggressive, leading to a less enjoyable playing experience. Despite the efforts of Square to make unique games with 3D features such as Rad Racer and 3-D Worldrunner, and high sales, the company was in financial trouble. These events are what led to a final attempt at a breakout hit, Final Fantasy. Rad Racer appeared in a scene in the movie The Wizard. It was also one of three games, including Super Mario Bros. and Tetris, featured at the 1990 Nintendo World Championship with an exclusive racing level to complete as one of the rounds of competition. The limited-edition release of the game's cartridge used in the tournament is now the rarest and most valuable Nintendo games available.",0.8908036351203918


### Testing Performance using IVF pq
Compresses the clusters and vectors, even faster than the above methods and slight trade off in accuarcy

In [33]:
%%sql
set @qv = (select v from vecs where id = 1125899906845489);

select paragraph, v <*> @qv as sim
from vecs
order by sim use index (ivf_pq) desc
limit 5;

5 rows affected.

paragraph,sim
"Rad Racer was ranked number 57 on IGN's Top 100 Nintendo Entertainment System games and was called ""iconic"" and one of the NES's premier racing games. Maxim included the game amongst thirteen others in their greatest 8-bit video games of all time list.",1.0000001192092896
"Rad Racer was met with favorable reviews, enjoyed commercial success, and sold 1.96 million copies. It also ranked 8th on Nintendo Power's player's poll Top 30. Famitsu praised the sense of speed but felt the game was slightly monotonous. Japanese publication Family Computer Magazine applauded the variety of game landscapes found in different levels. British magazine Computer and Video Games called it an ""extremely playable racing game"" and said ""things get very fast and competitive as you get further into the game.""",0.9292747974395752
"The game sold 1.96 million copies and is considered one of the best racing games on the NES, but was criticized as being derivative of other racing games from the period. Reviewers widely compared the game to Out Run, though opined that Rad Racer was different in some ways, and they praised the sense of speed. The game appeared in the 1989 film The Wizard and was one of three games to feature a unique competition course in the 1990 Nintendo World Championship.",0.9212067723274231
"Rad Racer was released in Japan on August 7, 1987. It was later released in North America in October 1987 and in Europe on January 15, 1988. In August 1987, Nintendo released an arcade machine called Playchoice-10 that contained up to ten popular NES titles. Amongst the eligible NES games, Rad Racer was also available on the arcade machine.",0.8954247236251831
"Rad Racer, known as Highway Star in Japan, is a racing video game developed and published by Square for the Nintendo Entertainment System in 1987. In this game, players drive a Ferrari 328 or a generic Formula One racing machine through a racecourse. The game was released in North America and Europe months after its debut. The title became well known for being one of two titles from Square that made use of stereoscopic 3D, which was made possible by wearing a pair of anaglyph glasses. Square president Masafumi Miyamoto initially conceived the game as an opportunity for developer Nasir Gebelli to demonstrate his 3D programming skills. Gebelli developed, and often drew by hand, the graphics for the game's 3D mode.",0.886330246925354


Power of Indexing - Used when working with large scale Vector data

## Hybrid Search in Single Store

In [34]:
%%sql

set @v_mario = (select v
                from vecs where url = "https://en.wikipedia.org/wiki/Super_Mario_Kart"
                order by id
                limit 1);

with fts as(
    select id, paragraph, match (paragraph) against ('Mario Kart') as score
    from vecs
    where match (paragraph) against ('Mario Kart')
    order by score desc
    limit 200
),
vs as (
    select id, paragraph, v <*> @v_mario as score
    from vecs
    order by score use index (auto) desc
    limit 200
)
select vs.id,
    vs.paragraph,
    .3 * ifnull(fts.score, 0) + .7 * vs.score as hybrid_score,
    vs.score as vec_score,
    ifnull(fts.score, 0) as ft_score
from fts full outer join vs
    on fts.id = vs.id
order by hybrid_score desc
limit 5;

5 rows affected.

id,paragraph,hybrid_score,vec_score,ft_score
1125899906848349,"Super Mario Kart is a kart racing game developed and published by Nintendo for the Super Nintendo Entertainment System . The first game in the Mario Kart series, it was released in Japan and North America in 1992, and in Europe the following year in 1993. Selling 8.76 million copies worldwide, the game went on to become the fourth best-selling SNES game of all time. Super Mario Kart was re-released on the Wii's Virtual Console in 2009, on the Wii U's Virtual Console in 2013, and on the New Nintendo 3DS's Virtual Console in 2016. Nintendo re-released Super Mario Kart in 2017 as part of the company's Super NES Classic Edition.",1.0000001668930054,1.000000238418579,1.0
2251799813690085,"Mario Kart DS was the best-selling game in its first month of release, and also held that position the following month. Overall, Mario Kart DS is the third best-selling game for the Nintendo DS as of March 2016, and the third best-selling Mario Kart game, behind Mario Kart 8 Deluxe and Mario Kart Wii, with 23.6 million units sold worldwide.",0.9238715887069702,0.8912451267242432,1.0
1125899906855047,"Mario Kart: Super Circuit is a 2001 kart racing game for the Game Boy Advance . It is the third Mario Kart game and retains its predecessors' gameplay: as a Mario franchise character, the player races opponents around tracks based on locales from the Super Mario platform games. Tracks contain obstacles and power-ups that respectively hamper and aid the player's progress. Super Circuit includes various single-player and multiplayer game modes, including a Grand Prix racing mode and a last man standing battle mode.",0.9178776502609253,0.8962398767471313,0.9683657884597778
1125899906848375,"Several sequels to Super Mario Kart have been released for successive generations of Nintendo consoles, each receiving commercial success and critical acclaim. While some elements have developed throughout the series, the core experience from Super Mario Kart has remained intact. The first sequel, Mario Kart 64, was released in 1996 for the Nintendo 64 and was the first Mario Kart game to feature fully 3D graphics. Although reviewers including IGN and GameSpot felt that the single-player gameplay was lacking compared to its predecessor, the simultaneous four-person multiplayer modes – a first for the Nintendo 64 – were praised. The second sequel, Mario Kart: Super Circuit, was released for the Game Boy Advance in 2001. It was described by GameSpot as more of a remake of Super Mario Kart than a sequel to Mario Kart 64 and featured a return to the graphical style of the original. As well as featuring new tracks, players are able to unlock the original SNES tracks if certain achievements are completed. Mario Kart: Double Dash was released for the GameCube in 2003. Unlike any other Mario Kart game before or since, it features two riders in each kart, allowing for a new form of cooperative multiplayer where one player controls the kart's movement and the other fires weapons. Mario Kart DS, released for the Nintendo DS in 2005, was the first Mario Kart game to include online play via the Nintendo Wi-Fi Connection. It went on to become the best selling handheld racing game of all time, selling 7.83 million units. The game marks the debut of tracks appearing in previous games. Mario Kart Wii was released for the Wii in 2008 and incorporates motion controls and 12-player racing. Like Mario Kart DS, it includes online play; it allows racers to play as user-created Miis as well as Mario series characters and comes packaged with the Wii Wheel peripheral, which can act as the game's primary control mechanism when coupled with a Wii Remote. Mario Kart Wii went on to be the worldwide best-selling game of 2008. Mario Kart 7 for the Nintendo 3DS was released in 2011, which features racing on land, sea, and air. Mario Kart 7 added the ability to customize the kart and to race in first-person mode. Three Mario Kart arcade ga

## Chating with Video Game Data
1) Use the power of semantic search & open AI language models to interact with video game data through chatbot
### Installing Necessary Libraries

In [39]:
!pip3 install openai --quiet

import sqlalchemy as sa
from openai import OpenAI
import getpass
import os
import time
import json

### Connecting OPEN AI API to Single Store
1) Setting Up Open AI API key and defining the connection
2) Specifing the models for generating the embeddings & interacting with GPT 3.5 Turbo
3) Will ask Input and store on the 'OPEN_API_KEY' environment variable

In [147]:
# OpenAI connection
OPENAI_API_KEY = os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')
client = OpenAI()
EMBEDDING_MODEL = 'text-embedding-ada-002'
GPT_MODEL = 'gpt-3.5-turbo'

# SingleStore connection
engine = sa.create_engine(connection_url)
connection = engine.connect()

OpenAI API Key:  ········


### Function to get embeddings 
Converts text input into Embedding Vector using openAI API which we use for semantic search 

In [80]:
def get_embedding(text, model=EMBEDDING_MODEL): 
    ''' Generates the Open AI embedding vector from an input `text`. '''
    if isinstance(text, str):
        response = client.embeddings.create(input=[text], model = model)
        return json.dumps(response.data[0].embedding)

### Function to perform semantic search
Perform semantic search from the database by the embedding vectors
Ranks and results the top results based on the similarity to the query

In [85]:
def search_wiki_page(query, limit=5):
    '''Returns a df of the top k matches to the query ordered by similarity.'''
    query_embedding_vec = get_embedding(query)
    statement = sa.text(
        f'''select paragraph, v <*> :query_embedding :> vector(1536) AS similarity
        from vecs
        order by similarity use index (auto) desc
        limit :limit;'''
    )
    print("Searching for matches...")
    start_time = time.time()
    results = connection.execute(statement, {"query_embedding": query_embedding_vec, "limit": limit})
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Search complete in {execution_time} seconds.")
    results_as_dict = results.fetchall()
    return results_as_dict

### Function to combine the above functions
Using RAG to answer the question from the wikipedia,using Semantic search the GPT chatbot gives the context aware response to the query

In [98]:
def ask_wiki_page(query, limit=5, temp=0.0):
    '''Uses RAG to answer a question from the wiki page'''
    results = search_wiki_page(query, limit)
    print("Asking Chatbot...")
    prompt = f'''Excerpt from the conversation history:
        {results}
        Question: {query}

        Based on the conversation history, try to provide the most accurate answer to the question.
        Consider the details mentioned in the conversation history to formulate a response that is as
        helpful and precise as possible. please provide links to WIKIPEDIA ARTICLES TO LOOK AT FOR MORE INFORMATION.

        Most importantly, IF THE INFORMATION IS NOT PRESENT IN THE CONVERSATION HISTORY, DO NOT MAKE UP AN ANSWER.'''
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {"role": "system", "content": "You are a helpful assistant who is answering questions about an article."},
            {"role": "user", "content": prompt}
        ],
        temperature=temp
    )
    response_message = response.choices[0].message.content
    return response_message

## Prompt the user to ask the questions

In [148]:
query = input('Ask me a question about video games!')
ask_wiki_page(query)

Ask me a question about video games! What do you know about RDR 2


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}